In [5]:
import multiSyncPy as msp
from multiSyncPy import synchrony_metrics as sm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from data_functions import get_ant_datafiles_info
from multisync_functions import add_multisync_metrics, get_multisync_statistics
from IPython.display import display


ant_colonies = get_ant_datafiles_info()
ant_colonies

,recording_location,file,location,n_ants,species_name,enterance
0,indoor,dataset/ants/IndoorDataset/Seq0001Object10Imag...,china,10,japanese_arched_ant,0
1,indoor,dataset/ants/IndoorDataset/Seq0002Object10Imag...,china,10,japanese_arched_ant,0
2,indoor,dataset/ants/IndoorDataset/Seq0003Object10Imag...,china,10,japanese_arched_ant,0
3,indoor,dataset/ants/IndoorDataset/Seq0004Object10Imag...,china,10,japanese_arched_ant,0
4,indoor,dataset/ants/IndoorDataset/Seq0005Object10Imag...,china,10,japanese_arched_ant,0
5,outdoor,dataset/ants/OutdoorDataset/Seq0006Object21Ima...,russia,73,carpenter_ant,1
6,outdoor,dataset/ants/OutdoorDataset/Seq0007Object26Ima...,greece,162,little_black_ant,1
7,outdoor,dataset/ants/OutdoorDataset/Seq0008Object31Ima...,russia,133,carpenter_ant,1
8,outdoor,dataset/ants/OutdoorDataset/Seq0009Object50Ima...,russia,193,carpenter_ant,1
9,outdoor,dataset/ants/OutdoorDataset/Seq0010Object30Ima...,usa,101,little_black_ant,0


In [6]:
original_multisync = add_multisync_metrics(ant_colonies)
display("ORIGNAL")
display(original_multisync.drop(columns=['file']))

surrogate_multisync = add_multisync_metrics(ant_colonies, surrogates=True)
display("SURROGATE")
display(surrogate_multisync.drop(columns=['file']))

'ORIGNAL'

,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic_entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.167,0.180,3.615,0.993
1,indoor,china,10,japanese_arched_ant,0,0.182,0.192,4.042,0.996
2,indoor,china,10,japanese_arched_ant,0,0.180,0.185,4.358,0.993
3,indoor,china,10,japanese_arched_ant,0,0.173,0.231,4.271,0.997
4,indoor,china,10,japanese_arched_ant,0,0.193,0.225,4.420,0.999
5,outdoor,russia,73,carpenter_ant,1,0.171,0.225,3.574,0.954
6,outdoor,greece,162,little_black_ant,1,0.173,0.216,3.569,0.953
7,outdoor,russia,133,carpenter_ant,1,0.172,0.204,3.740,0.995
8,outdoor,russia,193,carpenter_ant,1,0.198,0.298,3.211,0.918
9,outdoor,usa,101,little_black_ant,0,0.205,0.436,3.126,0.891


'SURROGATE'

,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic_entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.163,0.166,4.577,0.994
1,indoor,china,10,japanese_arched_ant,0,0.166,0.158,4.605,0.996
2,indoor,china,10,japanese_arched_ant,0,0.175,0.176,4.605,0.993
3,indoor,china,10,japanese_arched_ant,0,0.182,0.175,4.605,0.997
4,indoor,china,10,japanese_arched_ant,0,0.165,0.162,4.605,0.999
5,outdoor,russia,73,carpenter_ant,1,0.176,0.177,4.605,0.948
6,outdoor,greece,162,little_black_ant,1,0.188,0.186,4.605,0.955
7,outdoor,russia,133,carpenter_ant,1,0.173,0.179,4.605,0.994
8,outdoor,russia,193,carpenter_ant,1,0.173,0.173,4.605,0.918
9,outdoor,usa,101,little_black_ant,0,0.175,0.176,4.605,0.819


In [7]:
# surrogate T-test

import pandas as pd
from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic_entropy', 'rho']
results = []

for metric in metrics:
    original_metric = pd.to_numeric(original_multisync[metric], errors='coerce')
    surr_metric = pd.to_numeric(surrogate_multisync[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    results.append([metric, t_stat, p_val])

results_df = pd.DataFrame(results, columns=['Metric', 'T-Statistic', 'P-Value'])
results_df

,Metric,T-Statistic,P-Value
0,coherence,1.635604,0.119288
1,csd,2.717237,0.014125
2,symbolic_entropy,-5.543261,0.000029
3,rho,0.348545,0.731475


In [8]:
# compare indoor and outdoor ants
indoor = original_multisync[original_multisync['recording_location'] == 'indoor']
outdoor = original_multisync[original_multisync['recording_location'] == 'outdoor']

# by adding the synchrony measurements for each ant group, we can compare indoor and outdoor ants
results = get_multisync_statistics(indoor, outdoor, round_digits=4)
display(results)

,Metric,T-test (p-value),T-test (statistic),Mann-Whitney U (p-value),Mann-Whitney U (statistic)
0,coherence,0.5892,-0.5624,0.9166,11.5
1,csd,0.1391,-1.6428,0.1732,5.5
2,symbolic_entropy,0.0059,3.7165,0.0159,24.0
3,rho,0.0167,3.0140,0.0361,23.0
